##### 1. env에서 DB 정보 불러오고 연결 생성


In [3]:
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
from sqlalchemy.pool import QueuePool
import os

load_dotenv()

DATABASE_CONN = os.getenv("DATABASE_CONN")

In [4]:
engine = create_engine(
    DATABASE_CONN,  # echo=True,
    poolclass=QueuePool,
    # poolclass=NullPool, # Connection Pool 사용하지 않음.
    pool_size=10,
    max_overflow=0,
    pool_recycle=300,
)

##### 2. patterns_path 폴더내에서 패턴 정보가 포함된 .txt 파일들 모두 불러와서 정보 DB 삽입

- 선행적으로 행 정보 모두 삭제

- 다음으로 상, 중, 하, 윤곽 정보 db에 정보 삽입


In [6]:
# 테이블 모든 행 삭제
delete_query = "DELETE FROM shoes_data"

try:
    with engine.connect() as conn:
        conn.execute(text(delete_query))
        conn.commit()
        print("모든 행이 삭제되었습니다.")
except Exception as e:
    print(f"삭제 중 오류 발생: {e}")

모든 행이 삭제되었습니다.


In [16]:
from glob import glob
from tqdm import tqdm
import json

patterns_path = r"../data/P/*.txt"  # .txt 파일만 대상

patterns_info = sorted(
    glob(patterns_path), key=lambda x: int(x.split("\\")[-1].replace(".txt", ""))
)

query = """
INSERT INTO shoes_data (
image, model_number, top, mid, bottom, outline) VALUES (
    :image, :model_number, :top, :mid, :bottom, :outline
)
"""

for file_path in tqdm(patterns_info):
    model_number = file_path.split("\\")[-1].replace(".txt", "")

    with open(file_path, "r", encoding="CP949") as f:
        content = f.read().split()  # 파일 전체 읽기

    top, mid, bottom, outline = [], [], [], []
    i, pointer = 0, 1

    for partial in [top, mid, bottom, outline]:
        while i < len(content) and pointer < 5:
            if content[i].startswith(str(pointer)):
                partial.append(content[i][2:].strip())
                i += 1
            else:
                pointer += 1
                break

    # DB에 데이터 저장
    try:
        with engine.connect() as conn:
            conn.execute(
                text(query),
                {
                    'image': model_number,
                    "model_number": model_number,
                    "top": json.dumps(top) if top else json.dumps([]),
                    "mid": json.dumps(mid) if mid else json.dumps([]),
                    "bottom": json.dumps(bottom) if bottom else json.dumps([]),
                    "outline": json.dumps(outline) if outline else json.dumps([]),
                },
            )
            conn.commit()
    except Exception as e:
        print(f"Error processing {model_number}: {e}")
        break
else:
    print(f"모든 데이터가 성공적으로 삽입되었습니다.")

100%|██████████| 76673/76673 [06:25<00:00, 198.72it/s]

모든 데이터가 성공적으로 삽입되었습니다.
